In [ ]:
import pandas as pd
from prophet import Prophet
import plotly.graph_objects as go
import datetime
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from IPython.display import display # Optional: for better dataframe rendering in notebooks

# --- Configuration ---
DATA_PATH = r"E:\elevatetrsest\crop price predictor\Crop_price_Prediction\data\edited_21_24.csv"  # Make sure this file is in the same directory or provide the full path
TARGET_COLUMNS = ['avg_min_price', 'avg_max_price', 'avg_modal_price']
DATE_COLUMN = 'date'
MIN_DATA_POINTS = 30 # Minimum data points required to train a model

# --- User Selections (Replace Streamlit Sidebar Inputs) ---
# Set these values manually for your desired forecast
SELECTED_STATE = "Maharashtra"  # Example: Choose a state from your data
SELECTED_DISTRICT = "Akola"     # Example: Choose a district
SELECTED_COMMODITY = "Wheat"    # Example: Choose a commodity
FORECAST_DAYS = 90             # Example: Forecast period

# --- Data Loading Function ---
def load_data(path):
    """Loads and preprocesses the data."""
    try:
        df = pd.read_csv(path)
        print(f"Successfully loaded data from {path}")

        # Basic Preprocessing
        df[DATE_COLUMN] = pd.to_datetime(df[DATE_COLUMN], errors='coerce')
        initial_rows = len(df)
        df.dropna(subset=[DATE_COLUMN], inplace=True) # Drop rows where date conversion failed
        if initial_rows > len(df):
             print(f"Dropped {initial_rows - len(df)} rows due to invalid dates.")

        # Ensure price columns are numeric, coerce errors to NaN
        for col in TARGET_COLUMNS:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        # Optional: Impute missing prices if needed (example using ffill per group)
        # print("Attempting forward fill for missing prices within groups...")
        # df.sort_values([DATE_COLUMN,'state_name', 'district_name', 'commodity_name'], inplace=True)
        # df[TARGET_COLUMNS] = df.groupby(['state_name', 'district_name', 'commodity_name'])[TARGET_COLUMNS].ffill()

        initial_rows = len(df)
        df.dropna(subset=TARGET_COLUMNS, inplace=True) # Drop rows with missing target values
        if initial_rows > len(df):
             print(f"Dropped {initial_rows - len(df)} rows due to missing price data after preprocessing.")

        df.sort_values(DATE_COLUMN, inplace=True)
        print(f"Data preprocessing complete. {len(df)} rows remaining.")
        return df
    except FileNotFoundError:
        print(f"Error: Data file not found at {path}")
        return None
    except Exception as e:
        print(f"Error loading or preprocessing data: {e}")
        return None

# --- Modeling Function ---
def train_and_forecast(data, target_column, forecast_periods):
    """Trains a Prophet model and returns the model, forecast (starting from today),
       and predictions on the historical data for evaluation."""
    # Prepare data for Prophet (requires 'ds' and 'y' columns)
    prophet_df = data[[DATE_COLUMN, target_column]].rename(columns={DATE_COLUMN: 'ds', target_column: 'y'})

    # Check for sufficient data points for training
    if len(prophet_df) < MIN_DATA_POINTS:
        print(f"Warning: Not enough historical data points ({len(prophet_df)}) for '{target_column}' in the selected group to train. Need at least {MIN_DATA_POINTS}. Skipping forecast.")
        return None, None, None # Return None for model, forecast, and historical predictions

    try:
        print(f"\nTraining Prophet model for '{target_column}'...")
        # Instantiate and fit Prophet model on historical data
        model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=False, # Adjust based on expected patterns
            daily_seasonality=False   # Adjust based on expected patterns
        )
        model.fit(prophet_df) # Fit the model using historical data
        print("Model training complete.")

        # --- Create future dataframe STARTING FROM TODAY ---
        # Get today's date (midnight) based on system time when script runs
        current_date = pd.Timestamp.now().normalize()
        # Create a sequence of dates starting from today for the forecast period
        future_dates = pd.date_range(start=current_date, periods=forecast_periods, freq='D')
        future_df = pd.DataFrame({'ds': future_dates})
        # ----------------------------------------------------

        # Generate forecast using the dates starting from today
        print(f"Generating {forecast_periods}-day forecast starting from {current_date.strftime('%Y-%m-%d')}...")
        forecast = model.predict(future_df)
        print("Forecast generation complete.")

        # Generate predictions on historical data for evaluation
        historical_preds = model.predict(prophet_df) # Predict on the same data used for training

        return model, forecast, historical_preds # Return model, future forecast, historical predictions

    except Exception as e:
        print(f"Error during Prophet modeling or forecasting for {target_column}: {e}")
        return None, None, None

# --- Plotting Function for a Single Target ---
def plot_single_forecast(historical_data, forecast_data, target_column, title):
    """Creates a Plotly figure for one target's historical data and forecast."""
    fig = go.Figure()
    target_label = target_column.replace("avg_", "").replace("_price", "").capitalize() # Clean label for display

    # Add historical data trace
    hist_data_col = historical_data[[DATE_COLUMN, target_column]].dropna()
    fig.add_trace(go.Scatter(
        x=hist_data_col[DATE_COLUMN],
        y=hist_data_col[target_column],
        mode='lines',
        name=f'Historical {target_label}',
        line=dict(color='blue')
    ))

    # Add forecast trace (starts from today)
    fig.add_trace(go.Scatter(
        x=forecast_data['ds'],
        y=forecast_data['yhat'],
        mode='lines',
        name=f'Forecast {target_label}',
        line=dict(color='red', dash='dash')
    ))

    # Add uncertainty interval for the forecast
    fig.add_trace(go.Scatter(
        x=forecast_data['ds'],
        y=forecast_data['yhat_upper'],
        mode='lines', name='Forecast Upper Bound',
        line=dict(width=0),
        showlegend=False
    ))
    fig.add_trace(go.Scatter(
        x=forecast_data['ds'],
        y=forecast_data['yhat_lower'],
        mode='lines', name='Forecast Lower Bound',
        line=dict(width=0),
        fillcolor='rgba(255, 0, 0, 0.2)', # Light red fill for uncertainty
        fill='tonexty',
        showlegend=False
    ))

    fig.update_layout(
        title=title,
        xaxis_title='Date',
        yaxis_title=f'Price ({target_label})',
        hovermode="x unified",
        legend_title_text='Legend'
    )
    return fig

# --- Evaluation Metrics Function ---
def calculate_metrics(y_true, y_pred):
    """Calculates and returns R2, MAE, and MSE."""
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    return r2, mae, mse

# --- Main Execution Block ---
print("--- Crop Price Time Series Forecasting ---")
print(f"Forecasting from current date: {pd.Timestamp.now().normalize().strftime('%Y-%m-%d')}")

# Load data
df_full = load_data(DATA_PATH)

if df_full is not None:
    # --- Filtering Data Based on User Selections ---
    print(f"\nFiltering data for State='{SELECTED_STATE}', District='{SELECTED_DISTRICT}', Commodity='{SELECTED_COMMODITY}'...")
    filtered_df = df_full[
        (df_full['state_name'].str.strip().str.lower() == SELECTED_STATE.strip().lower()) &
        (df_full['district_name'].str.strip().str.lower() == SELECTED_DISTRICT.strip().lower()) &
        (df_full['commodity_name'].str.strip().str.lower() == SELECTED_COMMODITY.strip().lower())
    ].copy() # Use copy to avoid SettingWithCopyWarning

    # Ensure data is sorted by date (important for plotting historical correctly)
    filtered_df.sort_values(by=DATE_COLUMN, inplace=True)

    if filtered_df.empty:
        print("\nWarning: No historical data found for the selected combination.")
        print("Please check the CSV file and your selections (State, District, Commodity).")
    else:
        last_hist_date = filtered_df[DATE_COLUMN].max().strftime('%Y-%m-%d')
        print(f"\nFound {len(filtered_df)} historical data points (Latest: {last_hist_date}).")
        print(f"Proceeding with forecast for {FORECAST_DAYS} days.")

        all_forecasts = {} # Dictionary to store forecasts if needed later

        # Loop through each target price type
        for target in TARGET_COLUMNS:
            print("-" * 50)
            print(f"Processing Target: {target}")

            # Check if target column exists and has data after filtering
            if target not in filtered_df.columns or filtered_df[target].isnull().all():
                print(f"Warning: Target column '{target}' not found or contains only null values for the selection. Skipping.")
                continue

            # Prepare data for this specific target (dropping NaNs for this target)
            target_df = filtered_df[[DATE_COLUMN, target]].dropna().copy()
            if target_df.empty:
                 print(f"Warning: No valid data points for '{target}' after dropping NaNs. Skipping.")
                 continue

            # Train model, get forecast (future dates), and get historical predictions
            model, forecast, historical_preds = train_and_forecast(target_df, target, FORECAST_DAYS)

            if forecast is not None and historical_preds is not None:
                all_forecasts[target] = forecast # Store the forecast

                # --- Evaluate Model Fit on Historical Data ---
                print(f"\n--- Evaluating Model Fit for {target} (on historical data) ---")
                actuals = target_df[target] # Ground truth from the training data
                preds = historical_preds['yhat'] # Predictions on the training data
                
                # Ensure alignment - Prophet predictions match the input df length/order
                if len(actuals) == len(preds):
                    r2, mae, mse = calculate_metrics(actuals, preds)
                    print(f"R-squared (R2): {r2:.4f}")
                    print(f"Mean Absolute Error (MAE): {mae:.2f}")
                    print(f"Mean Squared Error (MSE): {mse:.2f}")
                else:
                    print("Warning: Mismatch between actuals and predictions length. Cannot calculate metrics accurately.")
                    print(f"Actuals length: {len(actuals)}, Predictions length: {len(preds)}")


                # --- Plot Historical Data and Forecast ---
                print(f"\n--- Plotting Historical Data & Forecast for {target} ---")
                plot_title = f'{target.replace("avg_", "").replace("_price", "").capitalize()} Price: Historical & {FORECAST_DAYS}-Day Forecast\n({SELECTED_COMMODITY} in {SELECTED_DISTRICT}, {SELECTED_STATE})'
                fig = plot_single_forecast(target_df, forecast, target, plot_title)
                fig.show() # Display the plot in the notebook output

                # --- Display Forecast Data Table ---
                print(f"\n--- Forecast Data Table for {target} ({FORECAST_DAYS} days) ---")
                f_display = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
                f_display.columns = ['Date', 'Forecast', 'Lower Bound', 'Upper Bound']
                f_display['Date'] = f_display['Date'].dt.strftime('%Y-%m-%d') # Format date
                # Use display for potentially nicer formatting in Jupyter, or just print
                display(f_display.set_index('Date').style.format("{:.2f}"))
                # Alternatively: print(f_display.set_index('Date').round(2).to_string())

            else:
                # Message already printed in train_and_forecast if skipped
                print(f"Skipping results display for {target} due to insufficient data or error during modeling.")

        print("-" * 50)
        print("\nForecasting process finished.")

else:
    print("\nFailed to load data. Cannot run the forecasting process.")

In [ ]:
import pandas as pd
from prophet import Prophet
import plotly.graph_objects as go
import datetime
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from IPython.display import display # Optional: for better dataframe rendering in notebooks

# --- Configuration ---
DATA_PATH = r"E:\elevatetrsest\crop price predictor\Crop_price_Prediction\data\edited_21_24.csv" # Use raw string for Windows paths
TARGET_COLUMNS = ['avg_min_price', 'avg_max_price', 'avg_modal_price']
DATE_COLUMN = 'date'
MIN_DATA_POINTS = 30 # Minimum data points required to train a model

# --- User Selections (Strings) ---
SELECTED_STATE_STR = "Maharashtra"
SELECTED_DISTRICT_STR = "Akola"
SELECTED_COMMODITY_STR = "Wheat"
FORECAST_DAYS = 90

# --- Frequency Encoding Mappings (CRITICAL: REPLACE WITH YOUR ACTUAL MAPPINGS) ---
# These dictionaries MUST map the original string names to the EXACT numerical
# values present in your 'edited_21_24.csv' file for these columns.
# Example placeholder - replace with your real mappings:
state_name_encoding_map = {
    "maharashtra": 1500, # Replace 1500 with the actual encoded value for Maharashtra
    "gujarat": 1200,
    # ... add all other states and their encoded values ...
}

district_name_encoding_map = {
    "akola": 50,    # Replace 50 with the actual encoded value for Akola
    "pune": 80,
    # ... add all other districts and their encoded values ...
}

commodity_name_encoding_map = {
    "wheat": 250,   # Replace 250 with the actual encoded value for Wheat
    "rice": 300,
    # ... add all other commodities and their encoded values ...
}
# --- End Mappings ---


# --- Data Loading Function (No changes needed here) ---
def load_data(path):
    """Loads and preprocesses the data."""
    try:
        df = pd.read_csv(path)
        print(f"Successfully loaded data from {path}")

        # Basic Preprocessing
        df[DATE_COLUMN] = pd.to_datetime(df[DATE_COLUMN], errors='coerce')
        initial_rows = len(df)
        df.dropna(subset=[DATE_COLUMN], inplace=True)
        if initial_rows > len(df):
             print(f"Dropped {initial_rows - len(df)} rows due to invalid dates.")

        # Ensure price columns are numeric, coerce errors to NaN
        for col in TARGET_COLUMNS:
            # Check if column exists before trying to convert
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            else:
                print(f"Warning: Target column '{col}' not found in the loaded CSV.")


        # --- Important: Assume state/district/commodity columns are already numeric ---
        # We no longer need to convert them, but we should check they exist
        filter_cols = ['state_name', 'district_name', 'commodity_name']
        for col in filter_cols:
            if col not in df.columns:
                 print(f"Error: Filtering column '{col}' not found in the loaded CSV. Cannot proceed.")
                 return None
             # Optional: Check if they are indeed numeric
             # if not pd.api.types.is_numeric_dtype(df[col]):
             #      print(f"Warning: Column '{col}' is expected to be numeric due to encoding, but it's not.")


        initial_rows = len(df)
        # Drop rows where ANY target column is missing
        df.dropna(subset=TARGET_COLUMNS, inplace=True, how='any')
        if initial_rows > len(df):
             print(f"Dropped {initial_rows - len(df)} rows due to missing price data in target columns.")

        df.sort_values(DATE_COLUMN, inplace=True)
        print(f"Data preprocessing complete. {len(df)} rows remaining.")
        return df
    except FileNotFoundError:
        print(f"Error: Data file not found at {path}")
        return None
    except Exception as e:
        print(f"Error loading or preprocessing data: {e}")
        return None

# --- Modeling Function (No changes needed here) ---
def train_and_forecast(data, target_column, forecast_periods):
    """Trains a Prophet model and returns the model, forecast (starting from today),
       and predictions on the historical data for evaluation."""
    # Prepare data for Prophet (requires 'ds' and 'y' columns)
    prophet_df = data[[DATE_COLUMN, target_column]].rename(columns={DATE_COLUMN: 'ds', target_column: 'y'})

    # Check for sufficient data points for training
    if len(prophet_df) < MIN_DATA_POINTS:
        print(f"Warning: Not enough historical data points ({len(prophet_df)}) for '{target_column}' in the selected group to train. Need at least {MIN_DATA_POINTS}. Skipping forecast.")
        return None, None, None # Return None for model, forecast, and historical predictions

    try:
        print(f"\nTraining Prophet model for '{target_column}'...")
        # Instantiate and fit Prophet model on historical data
        model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=False, # Adjust based on expected patterns
            daily_seasonality=False   # Adjust based on expected patterns
        )
        model.fit(prophet_df) # Fit the model using historical data
        print("Model training complete.")

        # --- Create future dataframe STARTING FROM TODAY ---
        current_date = pd.Timestamp.now().normalize()
        future_dates = pd.date_range(start=current_date, periods=forecast_periods, freq='D')
        future_df = pd.DataFrame({'ds': future_dates})
        # ----------------------------------------------------

        # Generate forecast using the dates starting from today
        print(f"Generating {forecast_periods}-day forecast starting from {current_date.strftime('%Y-%m-%d')}...")
        forecast = model.predict(future_df)
        print("Forecast generation complete.")

        # Generate predictions on historical data for evaluation
        historical_preds = model.predict(prophet_df) # Predict on the same data used for training

        return model, forecast, historical_preds # Return model, future forecast, historical predictions

    except Exception as e:
        print(f"Error during Prophet modeling or forecasting for {target_column}: {e}")
        return None, None, None

# --- Plotting Function (No changes needed here) ---
def plot_single_forecast(historical_data, forecast_data, target_column, title):
    """Creates a Plotly figure for one target's historical data and forecast."""
    fig = go.Figure()
    target_label = target_column.replace("avg_", "").replace("_price", "").capitalize() # Clean label for display

    # Add historical data trace
    hist_data_col = historical_data[[DATE_COLUMN, target_column]].dropna()
    fig.add_trace(go.Scatter(
        x=hist_data_col[DATE_COLUMN],
        y=hist_data_col[target_column],
        mode='lines',
        name=f'Historical {target_label}',
        line=dict(color='blue')
    ))

    # Add forecast trace (starts from today)
    fig.add_trace(go.Scatter(
        x=forecast_data['ds'],
        y=forecast_data['yhat'],
        mode='lines',
        name=f'Forecast {target_label}',
        line=dict(color='red', dash='dash')
    ))

    # Add uncertainty interval for the forecast
    fig.add_trace(go.Scatter(
        x=forecast_data['ds'],
        y=forecast_data['yhat_upper'],
        mode='lines', name='Forecast Upper Bound',
        line=dict(width=0),
        showlegend=False
    ))
    fig.add_trace(go.Scatter(
        x=forecast_data['ds'],
        y=forecast_data['yhat_lower'],
        mode='lines', name='Forecast Lower Bound',
        line=dict(width=0),
        fillcolor='rgba(255, 0, 0, 0.2)', # Light red fill for uncertainty
        fill='tonexty',
        showlegend=False
    ))

    fig.update_layout(
        title=title,
        xaxis_title='Date',
        yaxis_title=f'Price ({target_label})',
        hovermode="x unified",
        legend_title_text='Legend'
    )
    return fig

# --- Evaluation Metrics Function (No changes needed here) ---
def calculate_metrics(y_true, y_pred):
    """Calculates and returns R2, MAE, and MSE."""
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    return r2, mae, mse

# --- Main Execution Block ---
print("--- Crop Price Time Series Forecasting ---")
print(f"Forecasting from current date: {pd.Timestamp.now().normalize().strftime('%Y-%m-%d')}")

# Load data
df_full = load_data(DATA_PATH)

if df_full is not None:
    # --- Get Encoded Values for Filtering ---
    try:
        # Convert selected strings to lowercase to match typical map keys
        selected_state_key = SELECTED_STATE_STR.strip().lower()
        selected_district_key = SELECTED_DISTRICT_STR.strip().lower()
        selected_commodity_key = SELECTED_COMMODITY_STR.strip().lower()

        encoded_state = state_name_encoding_map.get(selected_state_key)
        encoded_district = district_name_encoding_map.get(selected_district_key)
        encoded_commodity = commodity_name_encoding_map.get(selected_commodity_key)

        # Check if lookup was successful
        if encoded_state is None:
            raise ValueError(f"State '{SELECTED_STATE_STR}' not found in state_name_encoding_map.")
        if encoded_district is None:
            raise ValueError(f"District '{SELECTED_DISTRICT_STR}' not found in district_name_encoding_map.")
        if encoded_commodity is None:
            raise ValueError(f"Commodity '{SELECTED_COMMODITY_STR}' not found in commodity_name_encoding_map.")

        print(f"\nSelected criteria (string): State='{SELECTED_STATE_STR}', District='{SELECTED_DISTRICT_STR}', Commodity='{SELECTED_COMMODITY_STR}'")
        print(f"Corresponding encoded values: State={encoded_state}, District={encoded_district}, Commodity={encoded_commodity}")

    except Exception as e:
        print(f"Error during mapping lookup: {e}")
        print("Please ensure the encoding maps are correct and contain the selected items.")
        df_full = None # Prevent further processing

if df_full is not None:
    # --- Filtering Data Based on Encoded Values ---
    print(f"\nFiltering data using encoded values...")
    filtered_df = df_full[
        (df_full['state_name'] == encoded_state) &
        (df_full['district_name'] == encoded_district) &
        (df_full['commodity_name'] == encoded_commodity)
    ].copy() # Use copy to avoid SettingWithCopyWarning

    # Ensure data is sorted by date (important for plotting historical correctly)
    filtered_df.sort_values(by=DATE_COLUMN, inplace=True)

    if filtered_df.empty:
        print("\nWarning: No historical data found for the selected combination using encoded values.")
        print("Please check the CSV file, your selections, and the encoding maps.")
    else:
        last_hist_date = filtered_df[DATE_COLUMN].max().strftime('%Y-%m-%d')
        print(f"\nFound {len(filtered_df)} historical data points (Latest: {last_hist_date}).")
        print(f"Proceeding with forecast for {FORECAST_DAYS} days.")

        all_forecasts = {} # Dictionary to store forecasts if needed later

        # Loop through each target price type
        for target in TARGET_COLUMNS:
             # First check if target column exists in the filtered data at all
             if target not in filtered_df.columns:
                 print(f"Target column '{target}' not present in the filtered data. Skipping.")
                 continue

             print("-" * 50)
             print(f"Processing Target: {target}")


             # Prepare data for this specific target (dropping NaNs for this target)
             # Ensure we only select existing columns DATE_COLUMN and target
             cols_to_select = [DATE_COLUMN, target]
             target_df = filtered_df[cols_to_select].dropna().copy() # Drop rows missing the target value

             if target_df.empty:
                 print(f"Warning: No valid data points for '{target}' after dropping NaNs for this specific target. Skipping.")
                 continue

             # Train model, get forecast (future dates), and get historical predictions
             model, forecast, historical_preds = train_and_forecast(target_df, target, FORECAST_DAYS)

             if forecast is not None and historical_preds is not None:
                 all_forecasts[target] = forecast # Store the forecast

                 # --- Evaluate Model Fit on Historical Data ---
                 print(f"\n--- Evaluating Model Fit for {target} (on historical data) ---")
                 actuals = target_df['y'].values # Get actual values from the prophet-formatted df
                 preds = historical_preds['yhat'].values # Get predictions

                 if len(actuals) == len(preds):
                     r2, mae, mse = calculate_metrics(actuals, preds)
                     print(f"R-squared (R2): {r2:.4f}")
                     print(f"Mean Absolute Error (MAE): {mae:.2f}")
                     print(f"Mean Squared Error (MSE): {mse:.2f}")
                 else:
                     print("Warning: Mismatch between actuals and predictions length. Cannot calculate metrics accurately.")
                     print(f"Actuals length: {len(actuals)}, Predictions length: {len(preds)}")


                 # --- Plot Historical Data and Forecast ---
                 print(f"\n--- Plotting Historical Data & Forecast for {target} ---")
                 plot_title = f'{target.replace("avg_", "").replace("_price", "").capitalize()} Price: Historical & {FORECAST_DAYS}-Day Forecast\n({SELECTED_COMMODITY_STR} in {SELECTED_DISTRICT_STR}, {SELECTED_STATE_STR})'
                 # Pass target_df which contains only date and the current target value
                 fig = plot_single_forecast(target_df, forecast, target, plot_title)
                 fig.show() # Display the plot in the notebook output

                 # --- Display Forecast Data Table ---
                 print(f"\n--- Forecast Data Table for {target} ({FORECAST_DAYS} days) ---")
                 f_display = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
                 f_display.columns = ['Date', 'Forecast', 'Lower Bound', 'Upper Bound']
                 f_display['Date'] = f_display['Date'].dt.strftime('%Y-%m-%d') # Format date
                 display(f_display.set_index('Date').style.format("{:.2f}"))

             else:
                 print(f"Skipping results display for {target} due to insufficient data or error during modeling.")

        print("-" * 50)
        print("\nForecasting process finished.")

else:
    print("\nFailed to load data or error during mapping lookup. Cannot run the forecasting process.")

In [5]:
import pandas as pd
from prophet import Prophet
import plotly.graph_objects as go
import datetime
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from IPython.display import display # Optional: for better dataframe rendering in notebooks
import numpy as np # Needed for IQR outlier check

# --- Configuration ---
# Use the RAW data path now
DATA_PATH = r"E:\elevatetrsest\crop price predictor\Crop_price_Prediction\data\wheat_price(2021-24).csv"
TARGET_COLUMNS = ['avg_min_price', 'avg_max_price', 'avg_modal_price']
DATE_COLUMN = 'date' # Ensure this matches the column name in your raw CSV
MIN_DATA_POINTS = 30 # Minimum data points required to train a model

# --- User Selections (Strings) ---
SELECTED_STATE_STR = "Maharashtra"  # Example: Choose a state from your data
SELECTED_DISTRICT_STR = "Akola"     # Example: Choose a district
SELECTED_COMMODITY_STR = "Wheat"    # Example: Choose a commodity
FORECAST_DAYS = 180            # Example: Forecast period

# --- Outlier Removal Function ---
def remove_outliers_iqr(df, columns_to_check):
    """Removes outliers from specified numerical columns using the IQR method."""
    df_filtered = df.copy()
    initial_rows = len(df_filtered)
    print(f"Applying IQR Outlier Removal on columns: {columns_to_check}")

    # Ensure columns exist and are numeric before calculating IQR
    valid_columns = [col for col in columns_to_check if col in df_filtered.columns and pd.api.types.is_numeric_dtype(df_filtered[col])]
    if not valid_columns:
        print("Warning: No valid numeric columns found for IQR outlier removal.")
        return df_filtered

    print(f"Valid numeric columns for IQR: {valid_columns}")
    subset_for_iqr = df_filtered[valid_columns]

    Q1 = subset_for_iqr.quantile(0.25)
    Q3 = subset_for_iqr.quantile(0.75)
    IQR = Q3 - Q1

    # Create a boolean mask for rows that are NOT outliers in ANY of the checked columns
    # Note: '~' negates the condition, keeping rows that are *within* the bounds.
    # .any(axis=1) flags a row if it's an outlier in *at least one* column.
    mask = ~((subset_for_iqr < (Q1 - 1.5 * IQR)) | (subset_for_iqr > (Q3 + 1.5 * IQR))).any(axis=1)

    df_filtered = df_filtered[mask]
    rows_removed = initial_rows - len(df_filtered)
    print(f"Removed {rows_removed} rows identified as outliers based on IQR.")
    return df_filtered


# --- Data Loading and Preprocessing Function (Incorporating User Steps) ---
def load_and_preprocess_data(path, date_col, target_cols):
    """Loads raw data and applies user-defined preprocessing steps."""
    try:
        print(f"Loading raw data from {path}...")
        df = pd.read_csv(path)
        print(f"Successfully loaded {len(df)} rows.")

        # --- User Preprocessing Steps START ---

        # 1. Handle Date Column (CRITICAL: Keep original date for Prophet)
        if date_col not in df.columns:
            print(f"Error: Date column '{date_col}' not found in CSV.")
            return None
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
        df.dropna(subset=[date_col], inplace=True) # Drop rows where date conversion failed
        print(f"{len(df)} rows after date parsing and dropping invalid dates.")

        # Optional: Extract day/month/year if needed for other analysis (but keep original date)
        # df['day'] = df[date_col].dt.day
        # df['month'] = df[date_col].dt.month
        # df['year'] = df[date_col].dt.year

        # 2. Initial dropna() as requested by user
        initial_rows = len(df)
        df.dropna(inplace=True)
        print(f"{len(df)} rows after initial broad dropna(). {initial_rows - len(df)} rows removed.")
        # Note: User's fillna('interpolate') was syntactically incorrect and likely redundant after dropna. Skipping.

        # 3. Ensure Price columns are numeric (Do this AFTER dropna potentially removes non-numeric entries)
        for col in target_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            else:
                print(f"Warning: Target column '{col}' not found.")
        # Drop rows where target columns became NaN after conversion (should be few if dropna worked)
        df.dropna(subset=target_cols, inplace=True, how='any')
        print(f"{len(df)} rows after ensuring target columns are numeric.")


        # 4. Frequency Encoding - SKIPPED HERE. Filtering needs original strings.
        #    If encoding is needed for a different model later, apply it AFTER filtering.

        # 5. Drop specified unused columns
        cols_to_drop = ['calculationType', 'district_id', 'change']
        # Check if columns exist before dropping
        existing_cols_to_drop = [col for col in cols_to_drop if col in df.columns]
        if existing_cols_to_drop:
            df.drop(columns=existing_cols_to_drop, axis=1, inplace=True)
            print(f"Dropped columns: {existing_cols_to_drop}")
        else:
             print(f"Columns to drop ({cols_to_drop}) not found.")


        # 6. Apply IQR Outlier Removal (Targeted at price columns)
        df = remove_outliers_iqr(df, target_cols)

        # --- User Preprocessing Steps END ---

        # Final check for required columns
        required_cols = [date_col] + target_cols + ['state_name', 'district_name', 'commodity_name']
        missing_req_cols = [col for col in required_cols if col not in df.columns]
        if missing_req_cols:
            print(f"Error: Required columns missing after preprocessing: {missing_req_cols}")
            return None

        df.sort_values(date_col, inplace=True)
        print(f"Preprocessing complete. {len(df)} rows remaining.")
        return df

    except FileNotFoundError:
        print(f"Error: Data file not found at {path}")
        return None
    except Exception as e:
        print(f"Error loading or preprocessing data: {e}")
        # print traceback for detailed debugging if needed
        # import traceback
        # traceback.print_exc()
        return None

# --- Modeling Function (No changes needed) ---
def train_and_forecast(data, target_column, forecast_periods):
    """Trains a Prophet model and returns the model, forecast (starting from today),
       and predictions on the historical data for evaluation."""
    # Prepare data for Prophet (requires 'ds' and 'y' columns)
    prophet_df = data[[DATE_COLUMN, target_column]].rename(columns={DATE_COLUMN: 'ds', target_column: 'y'})

    # Check for sufficient data points for training
    if len(prophet_df) < MIN_DATA_POINTS:
        print(f"Warning: Not enough historical data points ({len(prophet_df)}) for '{target_column}' in the selected group to train. Need at least {MIN_DATA_POINTS}. Skipping forecast.")
        return None, None, None # Return None for model, forecast, and historical predictions

    try:
        print(f"\nTraining Prophet model for '{target_column}'...")
        model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=False,
            daily_seasonality=False
        )
        model.fit(prophet_df)
        print("Model training complete.")

        current_date = pd.Timestamp.now().normalize()
        future_dates = pd.date_range(start=current_date, periods=forecast_periods, freq='D')
        future_df = pd.DataFrame({'ds': future_dates})

        print(f"Generating {forecast_periods}-day forecast starting from {current_date.strftime('%Y-%m-%d')}...")
        forecast = model.predict(future_df)
        print("Forecast generation complete.")

        historical_preds = model.predict(prophet_df)

        return model, forecast, historical_preds

    except Exception as e:
        print(f"Error during Prophet modeling or forecasting for {target_column}: {e}")
        return None, None, None

# --- Plotting Function (No changes needed) ---
def plot_single_forecast(historical_data, forecast_data, target_column, title):
    """Creates a Plotly figure for one target's historical data and forecast."""
    fig = go.Figure()
    target_label = target_column.replace("avg_", "").replace("_price", "").capitalize()

    # Add historical data trace (use DATE_COLUMN here)
    hist_data_col = historical_data[[DATE_COLUMN, target_column]].dropna()
    fig.add_trace(go.Scatter(
        x=hist_data_col[DATE_COLUMN],
        y=hist_data_col[target_column],
        mode='lines',
        name=f'Historical {target_label}',
        line=dict(color='blue')
    ))

    # Add forecast trace (uses 'ds' from forecast data)
    fig.add_trace(go.Scatter(
        x=forecast_data['ds'],
        y=forecast_data['yhat'],
        mode='lines',
        name=f'Forecast {target_label}',
        line=dict(color='red', dash='dash')
    ))

    # Add uncertainty interval for the forecast
    fig.add_trace(go.Scatter(
        x=forecast_data['ds'],
        y=forecast_data['yhat_upper'],
        mode='lines', name='Forecast Upper Bound',
        line=dict(width=0),
        showlegend=False
    ))
    fig.add_trace(go.Scatter(
        x=forecast_data['ds'],
        y=forecast_data['yhat_lower'],
        mode='lines', name='Forecast Lower Bound',
        line=dict(width=0),
        fillcolor='rgba(255, 0, 0, 0.2)',
        fill='tonexty',
        showlegend=False
    ))

    fig.update_layout(
        title=title,
        xaxis_title='Date',
        yaxis_title=f'Price ({target_label})',
        hovermode="x unified",
        legend_title_text='Legend'
    )
    return fig

# --- Evaluation Metrics Function (No changes needed) ---
def calculate_metrics(y_true, y_pred):
    """Calculates and returns R2, MAE, and MSE."""
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    return r2, mae, mse

# --- Main Execution Block ---
print("--- Crop Price Time Series Forecasting ---")
print(f"Forecasting from current date: {pd.Timestamp.now().normalize().strftime('%Y-%m-%d')}")

# Load data using the new preprocessing function
df_full = load_and_preprocess_data(DATA_PATH, DATE_COLUMN, TARGET_COLUMNS)

if df_full is not None:
    # --- Filtering Data Based on User Selections (using original string columns) ---
    print(f"\nFiltering data for State='{SELECTED_STATE_STR}', District='{SELECTED_DISTRICT_STR}', Commodity='{SELECTED_COMMODITY_STR}'...")

    # Check if filter columns exist before attempting to filter
    filter_cols = ['state_name', 'district_name', 'commodity_name']
    missing_filter_cols = [col for col in filter_cols if col not in df_full.columns]
    if missing_filter_cols:
        print(f"Error: Cannot filter because columns are missing: {missing_filter_cols}")
        df_full = None # Stop processing
    else:
        # Apply the filter using string methods (now valid)
        filtered_df = df_full[
            (df_full['state_name'].str.strip().str.lower() == SELECTED_STATE_STR.strip().lower()) &
            (df_full['district_name'].str.strip().str.lower() == SELECTED_DISTRICT_STR.strip().lower()) &
            (df_full['commodity_name'].str.strip().str.lower() == SELECTED_COMMODITY_STR.strip().lower())
        ].copy() # Use copy to avoid SettingWithCopyWarning

        # Ensure data is sorted by date after filtering
        filtered_df.sort_values(by=DATE_COLUMN, inplace=True)

# Proceed only if filtering was possible and resulted in data
if df_full is not None and 'filtered_df' in locals():

    if filtered_df.empty:
        print("\nWarning: No historical data found for the selected combination after preprocessing and filtering.")
        print("Please check the raw CSV file and your selections.")
    else:
        last_hist_date = filtered_df[DATE_COLUMN].max().strftime('%Y-%m-%d')
        print(f"\nFound {len(filtered_df)} historical data points for selection (Latest: {last_hist_date}).")
        print(f"Proceeding with forecast for {FORECAST_DAYS} days.")

        all_forecasts = {} # Dictionary to store forecasts if needed later

        # Loop through each target price type
        for target in TARGET_COLUMNS:
            if target not in filtered_df.columns:
                print(f"\nTarget column '{target}' not present in the filtered data. Skipping.")
                continue

            print("-" * 50)
            print(f"Processing Target: {target}")

            # Prepare data specific to this target (dropping NaNs ONLY for this target in the filtered data)
            target_df = filtered_df[[DATE_COLUMN, target]].dropna().copy()

            if target_df.empty:
                print(f"Warning: No valid data points remain for '{target}' after dropping NaNs within the filtered selection. Skipping.")
                continue

            # Train model, get forecast (future dates), and get historical predictions
            # Pass target_df which contains only date and the current target price
            model, forecast, historical_preds = train_and_forecast(target_df, target, FORECAST_DAYS)

            if forecast is not None and historical_preds is not None:
                all_forecasts[target] = forecast

                # --- Evaluate Model Fit on Historical Data ---
                print(f"\n--- Evaluating Model Fit for {target} (on historical data) ---")
                # Actual values are in the 'y' column of the data passed to Prophet
                prophet_input_df = target_df.rename(columns={DATE_COLUMN: 'ds', target: 'y'})
                actuals = prophet_input_df['y'].values
                preds = historical_preds['yhat'].values

                if len(actuals) == len(preds):
                    r2, mae, mse = calculate_metrics(actuals, preds)
                    print(f"R-squared (R2): {r2:.4f}")
                    print(f"Mean Absolute Error (MAE): {mae:.2f}")
                    print(f"Mean Squared Error (MSE): {mse:.2f}")
                else:
                    print("Warning: Mismatch between actuals and predictions length. Cannot calculate metrics accurately.")
                    print(f"Actuals length: {len(actuals)}, Predictions length: {len(preds)}")


                # --- Plot Historical Data and Forecast ---
                print(f"\n--- Plotting Historical Data & Forecast for {target} ---")
                plot_title = f'{target.replace("avg_", "").replace("_price", "").capitalize()} Price: Historical & {FORECAST_DAYS}-Day Forecast\n({SELECTED_COMMODITY_STR} in {SELECTED_DISTRICT_STR}, {SELECTED_STATE_STR})'
                # Pass target_df which contains the original date and target columns
                fig = plot_single_forecast(target_df, forecast, target, plot_title)
                fig.show()

                # --- Display Forecast Data Table ---
                print(f"\n--- Forecast Data Table for {target} ({FORECAST_DAYS} days) ---")
                f_display = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
                f_display.columns = ['Date', 'Forecast', 'Lower Bound', 'Upper Bound']
                f_display['Date'] = f_display['Date'].dt.strftime('%Y-%m-%d')
                display(f_display.set_index('Date').style.format("{:.2f}"))

            else:
                print(f"Skipping results display for {target} due to insufficient data or error during modeling.")

        print("-" * 50)
        print("\nForecasting process finished.")

elif df_full is None:
    print("\nFailed to load or preprocess data. Cannot run the forecasting process.")
# No 'else' needed here, covered by the check for filtered_df being empty or not existing

--- Crop Price Time Series Forecasting ---
Forecasting from current date: 2025-04-06
Loading raw data from E:\elevatetrsest\crop price predictor\Crop_price_Prediction\data\wheat_price(2021-24).csv...
Successfully loaded 29201 rows.
29201 rows after date parsing and dropping invalid dates.
29170 rows after initial broad dropna(). 31 rows removed.
29170 rows after ensuring target columns are numeric.
Dropped columns: ['calculationType', 'district_id', 'change']
Applying IQR Outlier Removal on columns: ['avg_min_price', 'avg_max_price', 'avg_modal_price']
Valid numeric columns for IQR: ['avg_min_price', 'avg_max_price', 'avg_modal_price']
Removed 1179 rows identified as outliers based on IQR.
Preprocessing complete. 27991 rows remaining.

Filtering data for State='Maharashtra', District='Akola', Commodity='Wheat'...

Found 1231 historical data points for selection (Latest: 2024-12-31).
Proceeding with forecast for 180 days.
--------------------------------------------------
Processing Tar

19:43:53 - cmdstanpy - INFO - Chain [1] start processing
19:43:53 - cmdstanpy - INFO - Chain [1] done processing


Model training complete.
Generating 180-day forecast starting from 2025-04-06...
Forecast generation complete.

--- Evaluating Model Fit for avg_min_price (on historical data) ---
R-squared (R2): 0.7481
Mean Absolute Error (MAE): 136.20
Mean Squared Error (MSE): 31542.04

--- Plotting Historical Data & Forecast for avg_min_price ---



--- Forecast Data Table for avg_min_price (180 days) ---


,Forecast,Lower Bound,Upper Bound
Date,,,
2025-04-06,2439.73,2214.24,2674.72
2025-04-07,2444.81,2222.96,2675.31
2025-04-08,2449.78,2227.95,2677.07
2025-04-09,2454.54,2236.56,2679.52
2025-04-10,2459.04,2238.85,2687.21
2025-04-11,2463.18,2233.14,2685.57
2025-04-12,2466.89,2242.12,2692.53
2025-04-13,2470.11,2225.21,2690.89
2025-04-14,2472.76,2245.62,2684.37


...

19:43:54 - cmdstanpy - INFO - Chain [1] done processing


Model training complete.
Generating 180-day forecast starting from 2025-04-06...
Forecast generation complete.

--- Evaluating Model Fit for avg_modal_price (on historical data) ---
R-squared (R2): 0.7801
Mean Absolute Error (MAE): 141.42
Mean Squared Error (MSE): 33848.73

--- Plotting Historical Data & Forecast for avg_modal_price ---



--- Forecast Data Table for avg_modal_price (180 days) ---


,Forecast,Lower Bound,Upper Bound
Date,,,
2025-04-06,2926.23,2690.74,3162.98
2025-04-07,2932.28,2699.01,3158.35
2025-04-08,2937.64,2708.59,3167.44
2025-04-09,2942.26,2697.38,3184.89
2025-04-10,2946.10,2726.04,3186.52
2025-04-11,2949.13,2722.79,3188.89
2025-04-12,2951.32,2710.14,3192.56
2025-04-13,2952.67,2697.24,3183.37
2025-04-14,2953.20,2701.58,3176.76


--------------------------------------------------

Forecasting process finished.
